In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import display
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.shape

(891, 12)

In [6]:
labels = train['Survived']
features_raw = train.drop(['Survived', 'Name', 'Ticket', 'PassengerId', 'Cabin'], axis=1)

features = pd.get_dummies(features_raw, columns = ['Pclass', 'Sex', 'Embarked'])

features['Age'].fillna(features['Age'].mean(), inplace=True)

features['Fare'].fillna(features['Fare'].mean(), inplace=True)
features['Fare'] /= (features['Fare'].max() - features['Fare'].min())

features = features.fillna(0)
print("features:")

features['Age'] = pd.cut(features['Age'], 5, labels=list(range(5)))
features = pd.get_dummies(features, columns = ['Age'])

display(features.head(6))

features:


,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_0,Age_1,Age_2,Age_3,Age_4
0,1,0,0.014151,0,0,1,0,1,0,0,1,0,1,0,0,0
1,1,0,0.139136,1,0,0,1,0,1,0,0,0,0,1,0,0
2,0,0,0.015469,0,0,1,1,0,0,0,1,0,1,0,0,0
3,1,0,0.103644,1,0,0,1,0,0,0,1,0,0,1,0,0
4,0,0,0.015713,0,0,1,0,1,0,0,1,0,0,1,0,0
5,0,0,0.016510,0,0,1,0,1,0,1,0,0,1,0,0,0


In [7]:
features.shape

(891, 16)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=4)

parameters = {'max_depth': [6, 8, 10, 12],
                  'min_samples_leaf': [4, 8, 10, 12],
                  'min_samples_split': [3, 5, 10]}

clf = DecisionTreeClassifier()
gs = GridSearchCV(clf, parameters)
gs.fit(X_train, y_train)

print("accuracy:",(gs.score(X_test, y_test)))

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


accuracy: 0.8491620111731844


In [112]:
def write_test(test_data, clf):

    features_raw = test_data.drop(['Name', 'Ticket', 'PassengerId', 'Cabin'], axis=1)
    features = pd.get_dummies(features_raw, columns = ['Pclass', 'Sex', 'Embarked'])

    features['Age'].fillna(features['Age'].mean(), inplace=True)

    features['Fare'].fillna(features['Fare'].mean(), inplace=True)
    features['Fare'] /= (features['Fare'].max() - features['Fare'].min())

    features = features.fillna(0)
    
    features['Age'] = pd.cut(features['Age'],5, labels=list(range(5)))
    features = pd.get_dummies(features, columns = ['Age'])

    pred = gs.predict(features)
    test_data.insert(1, 'Survived', pred)
    test_data = test_data.drop(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
                                'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
    test_data.to_csv('my_out.csv', index=False)

In [319]:
write_test(test, clf)